In [1]:
import pandas as pd
import numpy as np
import re 
from organizacao_fundos import df_tidy_simp

In [2]:
from unidecode import unidecode                   # só para fallback eventual
from organizacao_fundos import df_tidy_simp            # seu DataFrame financeiro
from mapear_codigo import ALIAS2CODE , CODIGOS_OFICIAIS, SUBCLASSES_OFICIAIS , normalizar

/Users/pedroauler/Desktop/MMZR-summer/limpeza.py:84: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_split = df_split.applymap(limpar_celula)


Ruído remanescente: 47.22%
                                                                  808
Promessa                                                           80
%                                                                  58
Obras                                                              56
Coobrigação                                                        52
Reserva                                                            48
coobrigação                                                        44
Reserva (02 PMTs)                                                  37
Obra                                                               35
cedente                                                            34
Reserva (03 PMTs)                                                  34
sócios pessoa física                                               31
jurídica                                                           27
direitos creditórios                                           

/Users/pedroauler/Desktop/MMZR-summer/limpeza.py:296: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  tokens_suspeitos = (gar_cols.stack(dropna=True)


In [ ]:
df_dash  = pd.read_excel("data/Estudo_de_Garantias_v3.xlsx",
                         sheet_name="Dashboard", header=1)

df_class = pd.read_excel("data/Estudo_de_Garantias_v3.xlsx",
                         sheet_name="Classificação", header=1)

# garantias já limpas/tokenizadas (colunas G1 … Gn)
df_tokens = pd.read_csv("data/garantias_limpas.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Estudo_de_Garantias_v3.xlsx'

In [17]:
df_fin   = df_tidy_simp.copy()   # mantém Norm., %PL etc.
df_all   = (
    df_fin.merge(df_tokens, on=["Fundo", "Ativo"], how="left")
           .reset_index(drop=True)
)

In [ ]:
df_class['Subclasse_norm'] = df_class['Subclasse'].apply(normalizar)

class_map = (df_class
             .dropna(subset=['Subclasse_norm'])
             .set_index(['Código', 'Subclasse_norm'])['Nota']
             .to_dict())

In [19]:
SUB_NORM2CANON = {normalizar(s): s for s in SUBCLASSES_OFICIAIS}

In [20]:
def classifica_token(tok: str):
    """
    Recebe um token bruto (já limpo) e devolve:
      - {"code": "AF"}              se for código ou alias de código
      - {"sub":  "Imóvel"}          se for subclasse oficial
      - {}                          caso contrário
    """
    if pd.isna(tok) or tok == "":
        return {}

    # 1) código oficial puro (duas ou três letras, etc.)
    if tok.upper() in CODIGOS_OFICIAIS:
        return {"code": tok.upper()}

    # 2) alias que mapeia para código
    tok_norm = normalizar(tok)
    if tok_norm in ALIAS2CODE:
        return {"code": ALIAS2CODE[tok_norm]}

    # 3) subclasse oficial
    if tok_norm in SUB_NORM2CANON:
        return {"sub": SUB_NORM2CANON[tok_norm]}

    # ruído
    return {}

In [21]:
def tokens_da_linha(row) -> tuple[list[str], list[str]]:
    """Extrai listas (codes, subs) a partir das colunas G1…Gn da linha."""
    codes, subs = [], []
    for col in row.index:
        if not col.startswith("G"):
            continue
        tok = row[col]
        info = classifica_token(tok)
        if "code" in info and info["code"] not in codes:
            codes.append(info["code"])
        if "sub"  in info and info["sub"]  not in subs:
            subs.append(info["sub"])
    return codes, subs

In [22]:
def nota_para_linha(codes: list[str], subs: list[str]) -> float:
    """Escolhe a melhor nota possível para a combinação codes × subs."""
    notas = []
    for c in codes:
        for s in subs:
            notas.append(class_map.get((c, s), np.nan))
    notas_validas = [n for n in notas if not np.isnan(n)]

    if notas_validas:
        return float(np.nanmax(notas_validas))

    # fallback: pega a melhor nota do código ignorando subclasse
    notas_code_only = [
        v for (cod, sub), v in class_map.items()
        if cod in codes and not pd.isna(v)
    ]
    if notas_code_only:
        return float(np.nanmax(notas_code_only))

    return np.nan

In [23]:
def calculo_score(norm , nota_calculada):
    produto = norm * nota_calculada
    soma = produto.sum()
    return soma /0.03

In [24]:
codes_and_subs = df_all.filter(like="G").columns      # salva a lista de colunas G*

df_all[["codes", "subs"]] = (
    df_all[codes_and_subs]
      .apply(tokens_da_linha, axis=1, result_type="expand")
)

df_all["Nota_calculada"] = df_all.apply(
    lambda r: nota_para_linha(r["codes"], r["subs"]), axis=1
)

In [25]:
scores = (
    df_all.groupby("Fundo", sort=False)[["Norm.", "Nota_calculada"]]
          .apply(lambda g: calculo_score(g["Norm."], g["Nota_calculada"]))
)


In [26]:
print("─── Scores por Fundo ───")
for fundo, val in scores.items():
    print(f"{fundo}: {val:.2f}")

─── Scores por Fundo ───
VGIR11: 92.41
MXRF11: 92.73
RBRY11: 96.03
KNCR11: 95.52
RBRR11: 93.10
CPTR11: 50.78
KNCA11: 50.36
RURA11: 1966.67
KNSC11: 90.45
BODB11: 93.11
KNUQ11: 61.91
HABT11: 98.45
